Imports

In [2]:
import numpy as np
import csv
from PIL import Image
import matplotlib.pyplot
from os import listdir
from os.path import isfile, join
import math

In [3]:
w = 101
h = 101

Reading in Training Data

In [4]:
import pickle
data = pickle.load(open("training_separated.p", 'rb'))
X = data[0]
y = data[1]

In [5]:
# Train Test Split
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20)

In [6]:
class PictureHelper():
    
    def __init__(self):
        self.i = 0

        self.training_images = X_train
        self.training_labels = y_train
        
        self.testing_images = X_test
        self.testing_labels = y_test
    
    def getTesting(self, batch_size):
        # shuffle data
        print(self.testing_images.shape) 
        print(self.testing_labels.shape)
        p = np.random.permutation(len(self.testing_images))
        self.testing_images = self.testing_images[p]
        self.testing_labels = self.testing_labels[p]   
        if (batch_size < self.testing_images.shape[0]):
            x = self.testing_images[:batch_size]
            y = self.testing_labels[:batch_size]
        else:
            x = self.testing_images
            y = self.testing_labels
        
        return x, y
        
        
    def next_batch(self, batch_size):
        
        if (self.i + batch_size >= self.training_images.shape[0]):
            p = numpy.random.permutation(len(self.training_images))
            self.training_images = self.training_images[p]
            self.training_labels = self.training_labels[p]
            
        x = self.training_images[self.i:self.i+batch_size]
        y = self.training_labels[self.i:self.i+batch_size]
            
        self.i = (self.i + batch_size) % len(self.training_images)
        
        return x, y

In [7]:
ph = PictureHelper()
x, y = ph.getTesting(10)
print(x.shape)
print(y.shape)

(1272, 101, 101, 1)
(1272, 2)
[922  47  50 ...   9 736  90]
(1272, 101, 101, 1)
(10, 101, 101, 1)
(10, 2)


## Creating the Model

** Import tensorflow **

In [8]:
import tensorflow as tf

In [9]:
x = tf.placeholder(tf.float32,shape=[None,w,h,1])
y_true = tf.placeholder(tf.float32,shape=[None,2])

In [10]:
hold_prob = tf.placeholder(tf.float32)

In [11]:
def init_weights(shape):
    init_random_dist = tf.truncated_normal(shape, stddev=0.1)
    return tf.Variable(init_random_dist)

def init_bias(shape):
    init_bias_vals = tf.constant(0.1, shape=shape)
    return tf.Variable(init_bias_vals)

def conv2d(x, W):
    return tf.nn.conv2d(x, W, strides=[1, 1, 1, 1], padding='SAME')

def max_pool_2by2(x):
    return tf.nn.max_pool(x, ksize=[1, 2, 2, 1],
                          strides=[1, 2, 2, 1], padding='SAME')

def convolutional_layer(input_x, shape):
    W = init_weights(shape)
    b = init_bias([shape[3]])
    return tf.nn.relu(conv2d(input_x, W) + b)

def normal_full_layer(input_layer, size):
    input_size = int(input_layer.get_shape()[1])
    W = init_weights([input_size, size])
    b = init_bias([size])
    return tf.matmul(input_layer, W) + b

# Creating Layers

In [12]:
convo_1 = convolutional_layer(x,shape=[4,4,1,32])
convo_2 = convolutional_layer(convo_1,shape=[4,4,32,128])
convo_2_pooling = max_pool_2by2(convo_2)

convo_3 = convolutional_layer(convo_2_pooling,shape=[4,4,128,256])
convo_4 = convolutional_layer(convo_3,shape=[4,4,256,256])
convo_4_pooling = max_pool_2by2(convo_4)

convo_5 = convolutional_layer(convo_4_pooling,shape=[4,4,256,128])
convo_6 = convolutional_layer(convo_5,shape=[4,4,128,128])
convo_6_pooling = max_pool_2by2(convo_6)

new_w = math.ceil(w / 8)
new_h = math.ceil(h / 8)

num_features = 128

convo_6_flat = tf.reshape(convo_6_pooling,[-1,new_h*new_w*num_features])

full_layer_1 = tf.nn.relu(normal_full_layer(convo_6_flat,1024))

full_layer_2 = tf.nn.relu(normal_full_layer(full_layer_1,1024))

full_dropout = tf.nn.dropout(full_layer_2,keep_prob=hold_prob)

y_pred = normal_full_layer(full_dropout,2)

cross_entropy = tf.nn.softmax_cross_entropy_with_logits(labels=y_true,logits=y_pred)

optimizer = tf.train.AdamOptimizer(learning_rate=0.001)
train = optimizer.minimize(cross_entropy)

init = tf.global_variables_initializer()

Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See @{tf.nn.softmax_cross_entropy_with_logits_v2}.



## Graph Session

** Perform the training and test print outs in a Tf session and run your model! **

In [13]:
gpu_options = tf.GPUOptions(per_process_gpu_memory_fraction=0.90)

In [14]:
from tensorflow.python.client import device_lib
device_lib.list_local_devices()

[name: "/device:CPU:0"
 device_type: "CPU"
 memory_limit: 268435456
 locality {
 }
 incarnation: 3481883938225486296, name: "/device:GPU:0"
 device_type: "GPU"
 memory_limit: 11280806708
 locality {
   bus_id: 1
   links {
   }
 }
 incarnation: 16244243147334921925
 physical_device_desc: "device: 0, name: Tesla K80, pci bus id: 0000:00:04.0, compute capability: 3.7"]

In [16]:
actual = []
predicted = []

In [ ]:
# config = tf.ConfigProto(gpu_options = gpu_options)
with tf.Session(config=tf.ConfigProto(log_device_placement=True)) as sess:
    sess.run(tf.global_variables_initializer())

    for i in range(1000):
        ph = PictureHelper()
        batch = ph.next_batch(8)
        sess.run(train, feed_dict={x: batch[0], y_true: batch[1], hold_prob: 1.0})
        # PRINT OUT A MESSAGE EVERY 100 STEPS
        print(i)
        if i%1 == 0:
            print('Currently on step {}'.format(i))
            print('Accuracy is:')
            # Test the Model
            x_test2, y_test2 = ph.getTesting(100)
            
            # metric for accuracy
            matches = tf.equal(tf.argmax(y_pred, axis = 1), tf.argmax(y_true, axis = 1))
            acc = tf.reduce_mean(tf.cast(matches,tf.float32))
            
            # generating the predictions
            temp_acc = sess.run(acc,feed_dict={x:x_test2, y_true:y_test2, hold_prob:1.0})

            print(temp_acc)
            if (i%10==0):
                predicted.append(sess.run(y_pred, feed_dict = {x:x_test2, y_true:y_test2, hold_prob:1.0}))
                actual.append(y_test2)

0
Currently on step 0
Accuracy is:
(1272, 101, 101, 1)
(1272, 2)
[ 677  523  171 ... 1187  387  319]
(1272, 101, 101, 1)
0.33
1
Currently on step 1
Accuracy is:
(1272, 101, 101, 1)
(1272, 2)
[1080  857  762 ...  191  710  167]
(1272, 101, 101, 1)
0.37
2
Currently on step 2
Accuracy is:
(1272, 101, 101, 1)
(1272, 2)
[942 632 711 ... 464 894 757]
(1272, 101, 101, 1)
0.6
3
Currently on step 3
Accuracy is:
(1272, 101, 101, 1)
(1272, 2)
[ 786  782  834 ... 1065  440  757]
(1272, 101, 101, 1)
0.55
4
Currently on step 4
Accuracy is:
(1272, 101, 101, 1)
(1272, 2)
[ 178  102  103 ...  321 1132  161]
(1272, 101, 101, 1)
0.34
5
Currently on step 5
Accuracy is:
(1272, 101, 101, 1)
(1272, 2)
[ 950  404  503 ...   34  732 1016]
(1272, 101, 101, 1)
0.41
6
Currently on step 6
Accuracy is:
(1272, 101, 101, 1)
(1272, 2)
[ 431  739  437 ...  300 1093 1105]
(1272, 101, 101, 1)
0.68
7
Currently on step 7
Accuracy is:
(1272, 101, 101, 1)
(1272, 2)
[1038  191   68 ...  380 1223  476]
(1272, 101, 101, 1)
0.56

0.59
65
Currently on step 65
Accuracy is:
(1272, 101, 101, 1)
(1272, 2)
[ 859  512  972 ...  833  360 1162]
(1272, 101, 101, 1)
0.67
66
Currently on step 66
Accuracy is:
(1272, 101, 101, 1)
(1272, 2)
[510 391 593 ... 127 818 612]
(1272, 101, 101, 1)
0.71
67
Currently on step 67
Accuracy is:
(1272, 101, 101, 1)
(1272, 2)
[ 847  487  245 ...  818  901 1139]
(1272, 101, 101, 1)
0.64
68
Currently on step 68
Accuracy is:
(1272, 101, 101, 1)
(1272, 2)
[1132  361  128 ...  538  125 1270]
(1272, 101, 101, 1)
0.57
69
Currently on step 69
Accuracy is:
(1272, 101, 101, 1)
(1272, 2)
[1099  349  138 ...  742   82  637]
(1272, 101, 101, 1)
0.39
70
Currently on step 70
Accuracy is:
(1272, 101, 101, 1)
(1272, 2)
[ 576  739  597 ... 1097  627 1250]
(1272, 101, 101, 1)
0.37
71
Currently on step 71
Accuracy is:
(1272, 101, 101, 1)
(1272, 2)
[ 376  519  843 ...  778  496 1221]
(1272, 101, 101, 1)
0.59
72
Currently on step 72
Accuracy is:
(1272, 101, 101, 1)
(1272, 2)
[355 845 946 ... 846 134 500]
(1272, 1

(1272, 101, 101, 1)
0.68
130
Currently on step 130
Accuracy is:
(1272, 101, 101, 1)
(1272, 2)
[ 271 1091  216 ...  403  178  438]
(1272, 101, 101, 1)
0.71
131
Currently on step 131
Accuracy is:
(1272, 101, 101, 1)
(1272, 2)
[ 928 1057  780 ...  677 1125 1219]
(1272, 101, 101, 1)
0.63
132
Currently on step 132
Accuracy is:
(1272, 101, 101, 1)
(1272, 2)
[1255  975  644 ...  880  946 1053]
(1272, 101, 101, 1)
0.73
133
Currently on step 133
Accuracy is:
(1272, 101, 101, 1)
(1272, 2)
[ 837  681 1107 ... 1068  994   85]
(1272, 101, 101, 1)
0.61
134
Currently on step 134
Accuracy is:
(1272, 101, 101, 1)
(1272, 2)
[1062  832 1228 ...  577 1187  172]
(1272, 101, 101, 1)
0.5
135
Currently on step 135
Accuracy is:
(1272, 101, 101, 1)
(1272, 2)
[ 754  423 1133 ...  103  947  554]
(1272, 101, 101, 1)
0.62
136
Currently on step 136
Accuracy is:
(1272, 101, 101, 1)
(1272, 2)
[1049  373  775 ... 1156 1269 1247]
(1272, 101, 101, 1)
0.61
137
Currently on step 137
Accuracy is:
(1272, 101, 101, 1)
(1272, 

(1272, 101, 101, 1)
0.61
194
Currently on step 194
Accuracy is:
(1272, 101, 101, 1)
(1272, 2)
[ 153  133  634 ... 1110  325  475]
(1272, 101, 101, 1)
0.62
195
Currently on step 195
Accuracy is:
(1272, 101, 101, 1)
(1272, 2)
[1107  484  774 ...  522  853  135]
(1272, 101, 101, 1)
0.63
196
Currently on step 196
Accuracy is:
(1272, 101, 101, 1)
(1272, 2)
[  80 1271  310 ...  921  993 1196]
(1272, 101, 101, 1)
0.58
197
Currently on step 197
Accuracy is:
(1272, 101, 101, 1)
(1272, 2)
[ 565  665  696 ... 1089  620  690]
(1272, 101, 101, 1)
0.65
198
Currently on step 198
Accuracy is:
(1272, 101, 101, 1)
(1272, 2)
[ 405   91 1040 ...  474  677   36]
(1272, 101, 101, 1)
0.64
199
Currently on step 199
Accuracy is:
(1272, 101, 101, 1)
(1272, 2)
[ 982  754 1119 ...  772  515   71]
(1272, 101, 101, 1)
0.64
200
Currently on step 200
Accuracy is:
(1272, 101, 101, 1)
(1272, 2)
[ 805 1073  787 ...  717  149  452]
(1272, 101, 101, 1)
0.59
201
Currently on step 201
Accuracy is:
(1272, 101, 101, 1)
(1272,

(1272, 101, 101, 1)
0.67
258
Currently on step 258
Accuracy is:
(1272, 101, 101, 1)
(1272, 2)
[1128  494  342 ...  145  957  599]
(1272, 101, 101, 1)
0.56
259
Currently on step 259
Accuracy is:
(1272, 101, 101, 1)
(1272, 2)
[ 994  817   61 ...  621  617 1198]
(1272, 101, 101, 1)
0.6
260
Currently on step 260
Accuracy is:
(1272, 101, 101, 1)
(1272, 2)
[1042   31 1244 ... 1189  808   40]
(1272, 101, 101, 1)
0.57
261
Currently on step 261
Accuracy is:
(1272, 101, 101, 1)
(1272, 2)
[636 899 278 ... 255 361 555]
(1272, 101, 101, 1)
0.59
262
Currently on step 262
Accuracy is:
(1272, 101, 101, 1)
(1272, 2)
[ 917  384  145 ... 1144  971  802]
(1272, 101, 101, 1)
0.61
263
Currently on step 263
Accuracy is:
(1272, 101, 101, 1)
(1272, 2)
[1233  227  921 ... 1026  449  855]
(1272, 101, 101, 1)
0.68
264
Currently on step 264
Accuracy is:
(1272, 101, 101, 1)
(1272, 2)
[502 512 453 ... 718 257 607]
(1272, 101, 101, 1)
0.55
265
Currently on step 265
Accuracy is:
(1272, 101, 101, 1)
(1272, 2)
[ 676  83

0.57
322
Currently on step 322
Accuracy is:
(1272, 101, 101, 1)
(1272, 2)
[ 853  964  720 ...  183  474 1170]
(1272, 101, 101, 1)
0.6
323
Currently on step 323
Accuracy is:
(1272, 101, 101, 1)
(1272, 2)
[ 554 1202  586 ...   93 1074  304]
(1272, 101, 101, 1)
0.56
324
Currently on step 324
Accuracy is:
(1272, 101, 101, 1)
(1272, 2)
[1142 1110  451 ...  707  606 1050]
(1272, 101, 101, 1)
0.65
325
Currently on step 325
Accuracy is:
(1272, 101, 101, 1)
(1272, 2)
[ 938 1271 1160 ...  838 1058  884]
(1272, 101, 101, 1)
0.59
326
Currently on step 326
Accuracy is:
(1272, 101, 101, 1)
(1272, 2)
[1156    4  573 ...  383   88  551]
(1272, 101, 101, 1)
0.55
327
Currently on step 327
Accuracy is:
(1272, 101, 101, 1)
(1272, 2)
[ 457 1054  418 ...  771  504  106]
(1272, 101, 101, 1)
0.62
328
Currently on step 328
Accuracy is:
(1272, 101, 101, 1)
(1272, 2)
[ 678  756 1022 ...  497  826  514]
(1272, 101, 101, 1)
0.57
329
Currently on step 329
Accuracy is:
(1272, 101, 101, 1)
(1272, 2)
[1018  852  702 .

# Analyzing Results

In [19]:
actual = np.array(actual)
predicted = np.array(predicted)
print(actual.shape)
print(predicted.shape)

(4, 100, 2)
(4, 100, 2)


In [ ]:
for i in range (100):
    print(actual[0][i], predicted[0][i])

In [ ]:
predicted[predicted<0.5] = 0
predicted[predicted>=0.5] = 1

In [62]:
import matplotlib.pyplot as plt

In [ ]:
i = 11
j = 70
plt.figure(0)
plt.imshow(predicted[i][j].reshape(101, 101), cmap = "gray")
plt.figure(1)
plt.imshow(actual[i][j].reshape(101, 101), cmap = "gray")

In [3]:
import pickle

In [4]:
data = pickle.load(open("TGS-Salt_results.p", 'rb'))
X = data[0]
y = data[1]

In [5]:
import matplotlib.pyplot as plt
%matplotlib inline

In [6]:
print(X.shape)
print(y.shape)

(5, 800, 10201)
(5, 800, 10201)


In [ ]:
batch = 4
for image in range (0, 50, 2):
    plt.figure(image)
    plt.imshow(X[batch][image].reshape(101, 101), cmap="gray")
    plt.figure(image+1)
    plt.imshow(y[batch][image].reshape(101, 101), cmap="gray")